# Query Expansion and Indexing

The following code is modified from Exercise 1. It is used to construct the vocabulary and vectorize the documents and query.

In [1]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import string
from nltk.corpus import stopwords
import math
from collections import Counter
nltk.download('stopwords')
import numpy as np

stemmer = PorterStemmer()

# Tokenize, stem a document
def tokenize(text):
    text = "".join([ch for ch in text if ch not in string.punctuation])
    tokens = nltk.word_tokenize(text)
    return " ".join([stemmer.stem(word.lower()) for word in tokens])

# Read a list of documents from a file. Each line in a file is a document
#with open("bread.txt") as f:
content = []
with open("epfldocs.txt") as f:
    content = f.readlines()
#original_documents = [x.decode('utf-8').strip() for x in content] # for python2
original_documents = [x.strip() for x in content] 
documents = [tokenize(d).split() for d in original_documents]

# create the vocabulary
vocabulary = set([item for sublist in documents for item in sublist])
vocabulary = [word for word in vocabulary if word not in stopwords.words('english')]
vocabulary.sort()

# compute IDF, storing idf values in a dictionary
def idf_values(vocabulary, documents):
    idf = {}
    num_documents = len(documents)
    for i, term in enumerate(vocabulary):
        idf[term] = math.log(num_documents/sum(term in document for document in documents), math.e)
    return idf

# Function to generate the vector for a document (with normalisation)
def vectorize(document, vocabulary, idf):
    vector = [0]*len(vocabulary)
    counts = Counter(document)
    max_count = counts.most_common(1)[0][1]
    for i,term in enumerate(vocabulary):
        vector[i] = idf[term] * counts[term]/max_count
    return vector

# Function to compute cosine similarity
def cosine_similarity(v1,v2):
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    if sumxy == 0:
            result = 0
    else:
            result = sumxy/math.sqrt(sumxx*sumyy)
    return result

def vectorize_query(query, vocabulary, idf):
    q = query.split()
    q = [stemmer.stem(w) for w in q]
    query_vector = vectorize(q, vocabulary, idf)
    return query_vector
    
def search_vec(query, k):
    query_vector = vectorize_query(query, vocabulary, idf)
    scores = [[cosine_similarity(query_vector, document_vectors[d]), d] for d in range(len(documents))]
    scores.sort(key=lambda x: -x[0])
    ans = []
    indices = []
    for i in range(min(k,len(original_documents))):
        ans.append(original_documents[scores[i][1]])
        indices.append(scores[i][1])
    return ans, indices, query_vector

# Compute IDF values and vectors
idf = idf_values(vocabulary, documents)
document_vectors = [vectorize(s, vocabulary, idf) for s in documents]

[nltk_data] Downloading package stopwords to /Users/elmas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Relevance Feedback

In this exercise we will implement and test Rocchio's method for user relevance feedback.

Let the set of relevant documents to be $D_r $ and the set of non-relevant documents to be $D_{nr}$. Then the modified query  $\vec{q_m}$  according to the Rocchio method is:

\begin{equation}
\vec{q_m} = \alpha \vec{q_0} + \frac{\beta}{|D_r|} \sum_{\vec{d_j} \in D_r} \vec{d_j} - \frac{\gamma}{|D_{nr}|} \sum_{\vec{d_j} \in D_{nr}} \vec{d_j}
\end{equation}
In the Rocchio algorithm negative term weights are ignored. This means, for the negative term weights in $\vec{q_m}$, we set them to be 0.

First, complete the implementation of the Rocchio relevance feedback method, by adding the missing code for the function $expand\_query$.   

Then, compare the result obtained with the new query with the unmodified one.

In [ ]:
def expand_query(relevant_doc_vecs, non_relevant_doc_vecs, query_vector, alpha, beta, gamma):
    
    num_rel = len(relevant_doc_vecs)
    num_non_rel = len(non_relevant_doc_vecs)
    
    # Compute the first term in the Rocchio equation
    norm_query_vector = ...
    
    # Compute the second term in the Rocchio equation
    norm_sum_relevant = ...
    
    # Compute the last term in the Rocchio equation
    norm_sum_non_relevant = ...
    
    # Sum all the terms
    modified_query_vector = ...
    
    # Ignore negative elements
    modified_query_vector = ...
    return modified_query_vector

In [ ]:
ans, result_doc_ids, query_vector = search_vec("computer science", 5)
for i in range(len(ans)):
    print(i,ans[i])

In [ ]:
# list of indices marked as relevant
# suppose first three documents were relevant and the rest were irrelevant.
relevant_indices = [0,1,2]
non_relevant_indices = [i for i in range(3, len(ans))]

relevant_doc_ids = [result_doc_ids[i] for i in relevant_indices]
non_relevant_doc_ids = [result_doc_ids[i] for i in non_relevant_indices]

relevant_doc_vecs = [document_vectors[i] for i in relevant_doc_ids]
non_relevant_doc_vecs = [document_vectors[i] for i in non_relevant_doc_ids]

expanded_query = expand_query(relevant_doc_vecs, non_relevant_doc_vecs, query_vector, 1, 1, 1)

new_query = ' '.join([vocabulary[i] for i, val in enumerate(expanded_query) if val>0])

new_ans , not_important_now, idontcare_anymore = search_vec(new_query, 10)

print('Modified query: ', new_query)
new_ans